In [229]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import glob, os, shutil

In [207]:
database_url = 'https://sonuc.ysk.gov.tr'
istanbul = 'İSTANBUL'

# Election names
ist_reelect = 'İSTANBUL BÜYÜKŞEHİR BELEDİYE BAŞKANLIĞI YENİLEME SEÇİMİ (23/06/2019)'
loc_2019 = '31 MART 2019 MAHALLİ İDARELER GENEL SEÇİMİ (31/03/2019)'
gen_2018 = 'CUMHURBAŞKANI VE 27.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (24/06/2018)'
ref_2017 = '2017 HALKOYLAMASI (16/04/2017)'
gen_nov_2015 = '26.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (01/11/2015)'
gen_jun_2015 = '25.DÖNEM MİLLETVEKİLİ GENEL SEÇİMİ (07/06/2015)'

In [208]:
# Creates a webdriver object for Chrome and navigates to Turkish high election board's election results database
chrome = webdriver.Chrome(executable_path='/Users/ercansen/Desktop/apps/chromedriver')
chrome.get(database_url)

In [209]:
# Website prompts the user to pick how to query the results; query by election name (third button) is selected
byElecNm_classname = 'ma-0.pa-0.border-0.cursor-pointer.w-100'

buttons_intro = list()
buttons_intro = chrome.find_elements_by_class_name(byElecNm_classname)
button_byElecNm = buttons_intro[2]
button_byElecNm.click()

In [210]:
# To open up the dropdown list of past 11 elections in Turkey, the corresponding arrow is clicked
elec_arrow_xpath = "//*[@id='collapsePanelThree']/div[1]/div/div/form/div[1]/div/div/ng-select/div/span"
elec_arrow = chrome.find_element_by_xpath(elec_arrow_xpath)

### Istanbul 2019 Mayoral Rerun Election

In [211]:
# Results of the rerun of 2019 Istanbul mayoral elections is scraped first, hence selected from the dropdown options
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == ist_reelect:
        option.click()
        break

In [212]:
# Green continue ("Devam Et") button is clicked
continue_classname = 'btn.btn-success.rounded-0.mr-5'

button_continue = chrome.find_element_by_class_name(continue_button_classname)
button_continue.click()

In [213]:
# Three dropdown lists (for filtering city, district and neighborhood name), and a filter for ballot box id appear
# (Note : District is the subdivision of city, and neighborhood is the subdivision of district)
# Due to database's design, when only a city is selected (but not a district) district-wise election results are shown
# When both a city and district are selected, results of each ballot box (in the district) are shown

# Corresponding arrow for city dropdown list is clicked
city_arrow_xpath = "//*[@id='collapsePanelFour']/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/div/span"
city_arrow = chrome.find_element_by_xpath(city_arrow_xpath)
city_arrow.click()

In [214]:
# Istanbul is selected from the list of cities (although it is the only option for the rerun mayoral election)
city_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[1]/ng-select/ng-dropdown-panel/div'
city_dropdown = chrome.find_element_by_xpath(city_dropdown_xpath)

for option in city_dropdown.find_elements_by_class_name('ng-option'):
    if option.text == istanbul:
        option.click()
        break

In [215]:
# Corresponding arrow for district dropdown list is selected
district_arrow_xpath = "//*[@id='collapsePanelFour']/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/span"
district_arrow = chrome.find_element_by_xpath(district_arrow_xpath)

In [227]:
district_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/ng-dropdown-panel/div'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

districts = list()
district_arrow.click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts.append(option.text)
    
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)   

In [243]:
downloads = '/Users/ercansen/Downloads/'
dest_dir  = '/Users/ercansen/Desktop/apps/istanbul/2019-06' 
os.mkdir(dest_dir)
files = glob.glob(downloads + 'Yurt*.xlsx')

for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, dest_dir)
        
for file in files:
    if os.path.isfile(file) and len(files) == 39:
        os.remove(file)

39


### 2019 Local Election (cancelled)

In [ ]:
# Results of the cancelled first run of 2019 Istanbul mayoral elections is scraped next
elec_arrow.click()

for option in chrome.find_elements_by_class_name('ng-option'):
    if option.text == loc_2019:
        option.click()
        break

In [ ]:
district_dropdown_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/ng-dropdown-panel/div'
query_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[2]/div/div/button'
district_input_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/fieldset/form/div[1]/div[2]/div[2]/ng-select/div/div/div[2]/input'
save_button_xpath = '/html/body/app-root/app-vatandas/div[2]/app-vatandas-asistan-arsiv-sonuc/div/div/div[2]/div[1]/div/div/app-yurtici-muhtarlik-listesi/fieldset/div[2]/div/button'
accept_button_xpath = '/html/body/ngb-modal-window/div/div/div[2]/div/button[1]'

query_button = chrome.find_element_by_xpath(query_button_xpath)
district_input = chrome.find_element_by_xpath(district_input_xpath)

districts = list()
district_arrow.click()
for option in chrome.find_element_by_xpath(district_dropdown_xpath).find_elements_by_class_name('ng-option'):
    districts.append(option.text)
    
for i in range(len(districts)):
    district = districts[i]
    time.sleep(3)
    district_input.send_keys(district)
    district_input.send_keys(Keys.ENTER)
    time.sleep(10)
    query_button.click()
    time.sleep(10)
    chrome.find_element_by_xpath(save_button_xpath).click()
    time.sleep(5)
    chrome.find_element_by_xpath(accept_button_xpath).click()
    time.sleep(5)  